In [2]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

abs_path = os.getcwd() + "/../../../"
abs_path = abs_path.replace("/", "\\") if os.name == "nt" else path
print(abs_path)


d:\Anaconda\envs\LLms\lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


g:\AI\AI-project\LLM-experiment\coder\SFT\rbt3\..\..\..\


In [3]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        path = abs_path + "dataset/rbt3/ChnSentiCorp_htl_all.csv"
        self.data = pd.read_csv(path)
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)
        

In [4]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


In [5]:
from torch.utils.data import random_split

train_set, valid_set = random_split(dataset, [0.9, 0.1])
print(len(train_set), len(valid_set))

6989 776


In [7]:
import torch 
print(abs_path + "coder\\model\\")
tokenizer = AutoTokenizer.from_pretrained("rbt3", cache_dir=abs_path + "coder/model/")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True)
    inputs["labels"] = torch.tensor(labels)
    return inputs
    

g:\AI\AI-project\LLM-experiment\coder\SFT\rbt3\..\..\..\coder/model/


In [15]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True, collate_fn=collate_func)
valid_loader = DataLoader(dataset=valid_set, batch_size=64, shuffle=True, collate_fn=collate_func)